In [1]:
import cunusht

ModuleNotFoundError: No module named 'cunusht'

In [1]:
import shtns

[SHTns 3.6.6] built May 27 2024, 07:48:01, id: v3.6.6-11-gf0a006a,avx512,ishioka,openmp


In [1]:
"""
Benchmark gclm2lenmap by scanning across different solvers, backends, and modes, and for different lmax values.
"""
import numpy as np
import time
import healpy as hp
import cunusht
import sys
from time import process_time

runinfos = [
    # ("CPU", "lenspyx"),
    # ("CPU", "duccnufft"),
    ("GPU", "cufinufft")
    ]
epsilons = [1e-08]
# lmaxs = [256*n-1 for n in np.arange(int(sys.argv[1]), 24)]
lmaxs = [256*int(10)-1]
runinfos = [("GPU", "cufinufft")] if 'GPU' == 'GPU' else [("CPU", "lenspyx")]
phi_lmaxs = [lmax for lmax in lmaxs]
defres = {}
Tsky = None
Tsky2 = None
nthreads = 10

for epsilon in epsilons:
    for runinfo in runinfos:
        for lmax, phi_lmax in zip(lmaxs, phi_lmaxs):
            geominfo = ('gl',{'lmax':lmax})
            lenjob_geominfo = ('gl',{'lmax':phi_lmax})
            lldlm = np.arange(0,phi_lmax+1)
            if False:
                from delensalot.sims.sims_lib import Xunl, Xsky
                synunl = Xunl(lmax=lmax, geominfo=geominfo, phi_lmax=phi_lmax)
                philm = synunl.get_sim_phi(0, space='alm')
                toydlm = hp.almxfl(philm, np.sqrt(lldlm*(lldlm+1)))
                toyunllm = synunl.get_sim_unl(0, spin=0, space='alm', field='temperature')
            else:
                nalm_unl = hp.Alm.getsize(lmax, mmax=lmax)
                toyunllm = np.array([np.random.rand(nalm_unl)*1e-6 + 1j*np.random.rand(nalm_unl)*1e-6])
                toydlm = np.random.rand(nalm_unl)*1e-6 + 1j*np.random.rand(nalm_unl)*1e-6

            backend = runinfo[0]
            defres.update({backend: {}}) if backend not in defres.keys() else None
            solver = runinfo[1]
            defres[backend].update({solver : None}) if solver not in defres[backend].keys() else None
            
            t = cunusht.get_transformer(backend, solver)
            if backend == 'CPU':
                if solver == 'lenspyx':
                    kwargs = {
                        'geominfo_deflection': lenjob_geominfo,
                        'dglm': toydlm,
                        'mmax_dlm': lmax,
                        'nthreads': nthreads,
                        'verbose': 1,
                        'epsilon': epsilon,
                        'single_prec': False,
                    }
                    t = t(**kwargs)
                    defres[backend][solver] = t.gclm2lenmap(
                            toyunllm.copy(), dlm=toydlm, lmax=lmax, mmax=lmax, spin=0, nthreads=10, execmode='timing', ptg=None)
                else:
                    kwargs = {
                        'geominfo_deflection': lenjob_geominfo,
                        'nuFFTtype': 2,
                    }
                    t = t(**kwargs)
                    defres[backend][solver] = t.gclm2lenmap(
                            gclm=toyunllm.copy(), dlm=toydlm, lmax=lmax, mmax=lmax, spin=0, nthreads=nthreads, epsilon=epsilon, execmode='timing', ptg=None)
            elif backend == 'GPU':
                import cupy as cp
                kwargs = {
                    'geominfo_deflection': lenjob_geominfo,
                    'epsilon': epsilon,
                    'nuFFTtype': 2,
                }
                t = t(**kwargs)
                lenmap = cp.empty(t.deflectionlib.constructor.spat_shape, dtype=cp.complex128).flatten()
                ll = np.arange(0,lmax+1,1)
                dlm_scaled = hp.almxfl(toydlm, np.nan_to_num(np.sqrt(1/(ll*(ll+1)))))
                dlm_scaled = cp.array(np.atleast_2d(dlm_scaled), dtype=np.complex128) #if kwargs['epsilon']<=1e-6 else cp.array(np.atleast_2d(dlm_scaled).astype(np.complex64))
                # defres[backend][solver] = t.gclm2lenmap(cp.array(toyunllm.copy()), dlm_scaled=dlm_scaled, lmax=lmax, mmax=lmax, lenmap=lenmap, ptg=None, execmode='timing', runid=int(sys.argv[4]))
                defres[backend][solver] = t.gclm2lenmap(toyunllm.copy(), dlm_scaled=dlm_scaled, lmax=lmax, mmax=lmax, lenmap=lenmap, ptg=None, execmode='timing', runid=int(100))


[SHTns 3.6.6] built May 27 2024, 07:48:01, id: v3.6.6-11-gf0a006a,avx512,ishioka,openmp,cuda
initializing shtns for CC in GPU_SHTns_transformer
geominfo for CC in GPU_SHTns_transformer:  ('cc', {'lmax': 2559, 'mmax': 2559, 'ntheta': 2592, 'nphi': 5120})
[SHTns 3.6.6] built May 27 2024, 07:48:01, id: v3.6.6-11-gf0a006a,avx512,ishioka,openmp,cuda
        Lmax=2559, Mmax*Mres=2559, Mres=1, Nlm=3278080  [10 threads, orthonormalized]
        > Condon-Shortley phase = 1, normalization = 0
!WARNING! Nlat must be larger than 2*Lmax for analysis to work (sampling theorem)!        => using FFTW : Mmax=2559, Nphi=5120, Nlat=2592, Nbatch=1  (theta-contiguous layout: phi_inc=2592, theta_inc=1)
          fftw cost ifftc=1.10653e+08,  fftc=1.10653e+08
          Memory used for Ylm and Zlm matrices = 97237.969 Mb x2
        => using Regular nodes including poles, with Clenshaw-Curtis quadrature
     !! Warning : Regular-Clenshaw-Curtis anti-aliasing condition Nlat > 2*Lmax is not met.
          Sum of

/tmp/ipykernel_2891921/1287995885.py:81: RuntimeWarning: divide by zero encountered in divide
  dlm_scaled = hp.almxfl(toydlm, np.nan_to_num(np.sqrt(1/(ll*(ll+1)))))


Running in timing execution mode
- - - - - - - - - - - - - - - Timing spin-1 synth: 0.260 seconds- - - - - - - - - - - - - - - 

- - - - - - - - - - - - - - - Timing pointing: 0.030 seconds- - - - - - - - - - - - - - - 

- - - - - - - - - - - - - - - Timing dlm2pointing: 0.295 seconds- - - - - - - - - - - - - - - 

- - - - - - - - - - - - - - - Timing dlm2pointing: 0.590 seconds- - - - - - - - - - - - - - - 



AttributeError: 'int' object has no attribute 'size'

In [3]:
import cupy as cp
a = cp.empty(10)

a.flags

b = np.empty(10)
b.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : True
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False